In [1]:
import pandas as pd

In [2]:
class Author:
    def __init__(self, first_name, last_name):
        self.first_name = first_name
        self.last_name = last_name

In [86]:
class Article:
    def __init__(self, 
                 article_id, 
                 title, 
                 publication, 
                 abstract, 
                 base64_file, 
                 publication_date,
                 volume, 
                 year, 
                 issue, 
                 page_number, 
                 section_title,
                 section_policy,
                 section_reference,
                 doi,
                 authors, 
                 locale):
        
        self.article_id = article_id
        self.title = title
        self.publication = publication
        self.abstract = abstract
        self.base64_file = base64_file
        self.publication_date = publication_date
        self.volume = volume
        self.year = year
        self.issue = issue
        self.page_number = page_number
        self.section_title = section_title
        self.section_policy = section_policy
        self.section_reference = section_reference
        self.doi = doi
        self.authors = authors
        self.locale = locale
    
    def export_authors(self):
        #generate a dict with authors and column titles
        amount_of_authors = len(self.authors)
        author_id = 0
        output = {}
        for a in self.authors:
            first_name_column = 'author_given_name_' + str(author_id)
            last_name_column = 'author_family_name_' + str(author_id)
            output[first_name_column] = [a.first_name]
            output[last_name_column] = [a.last_name]
            author_id += 1
        
        return output
    
    def to_row(self):
        #function that outputs the article as a single row for a df, as a list
        output = {'article_id': [self.article_id],
                 'title': [self.title],
                 'publication': [self.publication],
                'abstract': [self.abstract],
                'file': [self.base64_file],
                'publication_date': [self.publication_date],
                'volume': [self.volume],
                'year': [self.year],
                'issue': [self.issue],
                'page_number': [self.page_number],
                'section_title': [self.section_title],
                'section_policy': [self.section_policy],
                'section_reference': [self.section_reference],
                'doi': [self.doi]}
        
        authors = self.export_authors()
        
        output = output | authors
        
        return output

In [157]:
def get_article_info(article_node, root, article_id):
    
    base64_file = 'placeholder'
    
    publications = article_node.findall('{http://pkp.sfu.ca}publication')
    publication = publications[0]
    
    locale = publication.attrib['locale']
    publication_date = publication.attrib['date_published']
    section_reference = publication.attrib['section_ref']
    
    for id_node in publication.findall('{http://pkp.sfu.ca}id'):
        if id_node.get('type') == 'doi':  # Check for the 'type' attribute
            doi = id_node.text
    
    for title_node in publication.findall('{http://pkp.sfu.ca}title'):
        if title_node.get('locale') == locale:
            title = title_node.text
    
    for abstract_node in publication.findall('{http://pkp.sfu.ca}abstract'):
        if abstract_node.get('locale') == locale:
            abstract = abstract_node.text
    
    try:
        pages = publication.findall('{http://pkp.sfu.ca}pages')[0].text
    except IndexError:
        pages = ''

        
    author_list = publication.findall('.//{http://pkp.sfu.ca}author')
    authors = []
    for a in author_list:
        first_name = a.find('{http://pkp.sfu.ca}givenname').text
        last_name = a.find('{http://pkp.sfu.ca}familyname').text
        authors.append(Author(first_name, last_name))
        
    parent_issue = find_parent_issue(article_node, root)
    issue_identification = parent_issue.find('{http://pkp.sfu.ca}issue_identification')
    
    issue = issue_identification.find('{http://pkp.sfu.ca}number').text
    
    year = issue_identification.find('{http://pkp.sfu.ca}year').text
    
    for publication_node in issue_identification.findall('{http://pkp.sfu.ca}title'):
        if publication_node.get('locale') == locale:
            publication = publication_node.text
            
    section_information = parent_issue.find('{http://pkp.sfu.ca}sections')
    for section_node in section_information.findall('{http://pkp.sfu.ca}section'):
        if section_node.get('ref') == section_reference:
            for section_title_node in section_node.findall('{http://pkp.sfu.ca}title'):
                if section_title_node.get('locale') == locale:
                    section_title = section_title_node.text
                    
            for section_policy_node in section_node.findall('{http://pkp.sfu.ca}policy'):
                if section_policy_node.get('locale') == locale:
                    section_policy = section_policy_node.text
                    
    return Article(article_id, 
                 title, 
                 publication, 
                 abstract, 
                 base64_file, 
                 publication_date,
                 year, 
                 issue, 
                 page_number, 
                 section_title,
                 section_policy,
                 section_reference,
                 doi,
                 authors, 
                 locale)

In [158]:
get_article_info(articles[0], root, 1)

NameError: name 'volume' is not defined

In [53]:
import xml.etree.ElementTree as ET

In [74]:
namespaces = {
    'pkp': 'http://pkp.sfu.ca',  # Prefix 'pkp' mapped to namespace URL
    'xsi': 'http://www.w3.org/2001/XMLSchema-instance'
}

In [91]:
articles[0].

{'locale': 'en_US',
 'date_submitted': '2024-05-24',
 'status': '3',
 'submission_progress': '0',
 'current_publication_id': '1296',
 'stage': 'production'}

In [94]:
#an article has a publication
publications = articles[0].findall('{http://pkp.sfu.ca}publication')

In [96]:
publications[0].attrib

{'locale': 'en_US',
 'version': '1',
 'status': '3',
 'url_path': '',
 'seq': '2',
 'date_published': '2024-06-13',
 'section_ref': 'Film',
 'access_status': '0',
 '{http://www.w3.org/2001/XMLSchema-instance}schemaLocation': 'http://pkp.sfu.ca native.xsd'}

In [88]:
tree = ET.parse('test.xml')
root = tree.getroot()  # Get the root element

# Find all 'article' nodes using the namespace
articles = root.findall('.//{http://pkp.sfu.ca}article')

In [85]:
for article in articles:
    print(article.)

In [144]:
# Function to find the parent issue of a given article node
def find_parent_issue(article_node, root):
    for issue in root.findall('.//{http://pkp.sfu.ca}issue'):  # Iterate through all issues
        if article_node in issue.findall('.//{http://pkp.sfu.ca}article'):  # Check if the article is in this issue
            return issue
    return None

In [146]:
x = find_parent_issue(articles[0], root)

In [147]:
x.attrib

{'published': '1', 'current': '0', 'access_status': '1', 'url_path': ''}

In [143]:
root.findall('{http://pkp.sfu.ca}issue')

[<Element '{http://pkp.sfu.ca}issue' at 0x0000016B2FA26840>,
 <Element '{http://pkp.sfu.ca}issue' at 0x0000016B2FA2F600>]